In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_data = pd.read_csv("train-data.tsv",sep='\t', header= None, names=['label', 'message'])
train_data.head()
valid_data = pd.read_csv("valid-data.tsv",sep='\t', header= None, names=['label', 'message'])
valid_data.head()

In [ ]:
label_encoder_class = LabelEncoder()
train_data['label'] = label_encoder_class.fit_transform(train_data['label'])
valid_data['label'] = label_encoder_class.fit_transform(valid_data['label'])

train_labels = train_data.pop("label")
valid_labels = valid_data.pop("label")

print(train_labels)

In [ ]:
encoder = keras.layers.TextVectorization()
encoder.adapt(train_data)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,), dtype=tf.string),
    encoder,
    tf.keras.layers.Embedding(
    input_dim=len(encoder.get_vocabulary()),
    output_dim=64,
    mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
# Stopping condition
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                  mode="min",
                                  patience=3, #stops when doesnt improve for 3 consective epochs
                                  restore_best_weights=True)

history = model.fit(x=train_data,y=train_labels, epochs=20,
                    validation_data=(valid_data,valid_labels),
                    callbacks=[callback])


In [ ]:
 # function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
threshold = 0.5
prediction = ""
def predict_message(pred_text):
  class_prediction = model.predict(np.array([pred_text], dtype=object))
  predicted_classes = (class_prediction > threshold).astype(int)
  if predicted_classes == 0:
    prediction = [class_prediction, "ham"]
  else:
    prediction = [class_prediction, "spam"]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
